In [2]:
%matplotlib inline

from dit.divergences import jensen_shannon_divergence
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from dateutil.parser import parse
import plotly.graph_objs as go
from shapely.geometry import *
from datetime import datetime  
import plotly.plotly as py
import scipy.stats as st
import geopandas as gpd
from scipy import stats
import pandas as pd
import numpy as np  
import operator
import geojson
import math 
import sys 
import dit

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# data.london.gov.uk/dataset/statistical-gis-boundary-files-london 
singapore = gpd.read_file('../data/MP14_SUBZONE_WEB_PL.shp')
# from the prj file: http://www.prj2epsg.org/epsg/27700
singapore = singapore.to_crs({'init': 'epsg:4277'})   

# path to data
venues = "../../../raw_data/venues/Singapore_venues.txt"
transitions = "../../../raw_data/transitions/Singapore_transitions.txt"

# venue ID -> name
venue_id_to_name = {} 
# venue ID -> coordinates for that venue
venue_id_to_coords = {} 
# venue ID -> SPECIFIC category
venue_id_to_category_specific = {} 
# venue ID -> GENERAL category
venue_id_to_category_general = {} 
# venue ID -> opening date
venue_id_to_opening = {}  

### STEP 1: get all venues 
with open(venues, 'r') as f:
    for rows in f:
        rows = rows.split("\t")
        # unique ID 
        ID = rows[0] 
        # name
        name = rows[1]
        venue_id_to_name[ID] = name 
        # coordinates 
        coordinates = (float(rows[2]), float(rows[3]))
        venue_id_to_coords[ID] = coordinates  
        # specific category
        type_venue = rows[7]
        venue_id_to_category_specific[ID] = type_venue  
        # general categorgy
        type_venue = rows[8]
        venue_id_to_category_general[ID] = type_venue  
        
        # date
        date = rows[9].strip()  
        date_cleaned = datetime.strptime(date, '%Y-%m-%d')
        venue_id_to_opening[ID] = date_cleaned
        

LIST_OF_VENUES = venue_id_to_opening.keys()

# venue ID -> list of *FIRST* checkins to that venue       
venue_id_to_times = {} 
with open(transitions, 'r') as f:
    for rows in f:
        rows = rows.split("\t")
        location1 = rows[0]
        time1 = rows[1] 
        if location1 in venue_id_to_coords:
            venue_id_to_times.setdefault(location1, [])
            time_cleaned = datetime.fromtimestamp(int(time1))
            venue_id_to_times[location1].append(time_cleaned)
             
# venue id to the number of checkins to that venue 
venue_id_to_num = {}
for v in LIST_OF_VENUES:
    if v not in venue_id_to_times:
        continue 
    venue_id_to_num[v] = len(venue_id_to_times[v])    
    
# venue ID -> ward
venue_id_to_ward = {}
for venue in LIST_OF_VENUES: 
    coordinates = venue_id_to_coords[venue]
    latitude, longitude = coordinates
    point_of_checkin = Point(longitude, latitude) 
    
    does_contain_point = singapore.contains(point_of_checkin) 
    contains = does_contain_point[does_contain_point == True]      
    if len(contains) > 0:
        ward = contains.index[0] 
        venue_id_to_ward[venue] = ward 

# ward to set of venues in that ward
ward_to_set_venues = {} 
# venue id to the number of checkins to that venue 
venue_id_to_num = {}
for v in LIST_OF_VENUES:
    if v not in venue_id_to_times:
        continue 
    venue_id_to_num[v] = len(venue_id_to_times[v])
    
    if v not in venue_id_to_ward: 
        continue 
    ward = venue_id_to_ward[v]
    ward_to_set_venues.setdefault(ward, set())
    ward_to_set_venues[ward].add(v)

Using TensorFlow backend.


IOError: no such file or directory: '../data/MP14_SUBZONE_WEB_PL.shp'

In [ ]:
### STEP 2: get all new venues
# get a set of places that opened after the day we've selected  
places_opened_to_num = {}
for v in venue_id_to_opening: 
    if v not in venue_id_to_num: 
        continue 
    date = venue_id_to_opening[v]
    year = date.year
    month = date.month
    if (year > 2011 and month > 6) or year > 2012: 
        places_opened_to_num[v] = venue_id_to_num[v]
          
# list of new venues that have at least 100 checkins 
venues_above_threshold = [venue for venue in places_opened_to_num if places_opened_to_num[venue] > 100]
print len(venues_above_threshold)

In [ ]:
# check if there's data last 6 months
# is there less than 12 checkins after july 2013
venues_closed = set()
# whether venue is open or closed 
venue_to_status = {}
for v in venues_above_threshold:
    if v not in venue_id_to_times: 
        continue
    times = venue_id_to_times[v]
    checkins_after = 0
    for date in times:
        year = date.year
        month = date.month
        if year == 2013 and month > 6 or year > 2014:
            checkins_after += 1 
    if checkins_after < 50: # less than 5 checkins a month
        venues_closed.add(v)     
        venue_to_status[v] = 0
    else: 
        venue_to_status[v] = 1 

In [14]:
num_closed = 0

date_comparison = datetime.strptime('Jul2013', '%b%Y')
max_possible = datetime.strptime('Jun2011', '%b%Y')   
max_possible = date_comparison - max_possible
arr_days_open = []

for v in venues_above_threshold: 
    if venue_to_status[v] == 0:
        num_closed += 1 
        delta = date_comparison - venue_id_to_opening[v]
        arr_days_open.append(delta.days)
        #print venue_id_to_opening[v], " - ", date_comparison, " - ", delta 
    
print max_possible
print len(arr_days_open)

761 days, 0:00:00
94


In [15]:
# when people end up at this ward, from where do they come?
# that same ward or a different ward?
ward_to_num_inflow_transitions = {}
ward_to_num_within_transitions = {}

# when people leave this ward, where do they go? 
# that same ward or a different ward?
ward_to_num_outflow_transitions = {} 
error_counter = 0
with open(transitions, 'r') as f:
    for rows in f:
        rows = rows.split("\t")
        location_1 = rows[0]
        location_2 = rows[2]
        if location_1 not in venue_id_to_ward or location_2 not in venue_id_to_ward: 
            error_counter += 1
            continue
        ward_1 = venue_id_to_ward[location_1]
        ward_2 = venue_id_to_ward[location_2]
        if ward_1 == ward_2:
            ward_to_num_within_transitions.setdefault(ward_2, 0)
            ward_to_num_within_transitions[ward_2] += 1
        else:  
            ward_to_num_inflow_transitions.setdefault(ward_2, 0)
            ward_to_num_inflow_transitions[ward_2] += 1 
            
            ward_to_num_outflow_transitions.setdefault(ward_1, 0)
            ward_to_num_outflow_transitions[ward_1] += 1

In [16]:
all_general = list(set(venue_id_to_category_general.values()))
all_specific = list(set(venue_id_to_category_specific.values()))

myData = np.array([["venue", "ward", "within", "in", "out", "ratio in", "ratio out", "ratio in/out", "spec", "gen", "closed or open"]])
NUM_FEATURES = 10
count = 0
for new_venue in venues_above_threshold: 
    # travel out of city bounds 
    if new_venue not in venue_id_to_ward:  
        continue 
    inner= np.array([])
    new_ward = venue_id_to_ward[new_venue] 
    if new_ward not in ward_to_num_within_transitions or new_ward not in ward_to_num_inflow_transitions:  
        print new_ward
        continue
    if new_ward not in ward_to_num_outflow_transitions or new_venue not in venue_to_status: 
        continue 
    if new_venue not in venue_id_to_category_specific or new_venue not in venue_id_to_category_general: 
        continue
         
    within = ward_to_num_within_transitions[new_ward]
    inflow = ward_to_num_inflow_transitions[new_ward]
    outflow = ward_to_num_outflow_transitions[new_ward]
    status = venue_to_status[new_venue]
    spec = venue_id_to_category_specific[new_venue]
    gen = venue_id_to_category_general[new_venue]
    index_spec = all_specific.index(spec)
    index_gen = all_general.index(gen) 
    
    
    inner = np.append(inner, count)
    inner = np.append(inner, new_ward)
    inner = np.append(inner, within)
    inner = np.append(inner, inflow)
    inner = np.append(inner, outflow)    
    inner = np.append(inner, float(within) / inflow)
    inner = np.append(inner, float(within) / outflow)
    inner = np.append(inner, float(inflow) / outflow)
    inner = np.append(inner, index_spec)
    inner = np.append(inner, index_gen)
    
    # output
    inner = np.append(inner, status)  
    myData = np.concatenate([myData, [inner]])   
    count += 1

In [17]:
count

378

In [18]:
import csv
myFile = open('features.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(myData)  
print("Writing complete")

Writing complete


In [19]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [21]:
dataframe = pandas.read_csv("features.csv", header=None)
dataset = dataframe.values 

X = dataset[:, 0:NUM_FEATURES]
Y = dataset[:, NUM_FEATURES]

def create_baseline(): 
    model = Sequential()
    model.add(Dense(1, input_dim=NUM_FEATURES, kernel_initializer='normal', activation='relu')) # intially 60
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid')) 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model  

estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) 

Results: 70.54% (14.52%)


In [ ]:
# step 3: train model
# https://www.tensorflow.org/get_started/get_started
# https://keras.io/
# https://stackoverflow.com/questions/42532386/how-to-work-with-multiple-inputs-for-lstm-in-keras
# https://github.com/titu1994/keras-SRU/blob/master/imdb_sru.py

# https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/
# https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/



# MAIN
# https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/

# 
# https://keras.io/layers/core/

# 30 - Results: 70.53% (28.97%) 
# 10 - Results: 78.88% (21.96%)
# 1 - Results: 86.51% (0.78%)